Unigram Model

In [2]:
import sentencepiece as spm
from collections import Counter

# Define the path to save the trained Unigram model
model_prefix = "hindi_unigram_model_Unigram"

# Load the Hindi corpus
corpus_file = "hi_100_1.txt"

# Train the Unigram model
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, model_type='unigram', character_coverage=1.0)
# Load the trained Unigram model
unigram_model = spm.SentencePieceProcessor()
unigram_model.load(f"{model_prefix}.model")





True

In [3]:
# First Paramter is file path  so please specify the file path
with open('hi_100_1.txt', "r", encoding="utf-8") as f:
    corpus = f.read()

tokenized_corpus = unigram_model.encode(corpus, out_type=str)



for i, token in enumerate(tokenized_corpus):
    if token.startswith("▁"):
        tokenized_corpus[i] = token[1:].replace("▁", "")
        
# for i in tokenized_corpus:
#     print(i)



# Count the frequency of each token
token_counter = Counter(tokenized_corpus)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)

i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1

के - 328358
। - 276966
में - 240967
है - 229106
की - 200828
, - 187641
को - 155678
से - 147909
. - 127557
का - 127290
ने - 126362
और - 115827
पर - 98012
कि - 83604
ी - 78808
न - 67903
हैं - 66265
भी - 66242
कर - 65465
ों - 61200


In [10]:
def unic(text):
    actual_matras = ['ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो', 'ौ', 'ृ', 'ॅ', 'ॉ', 'ं', 'ः', 'ँ']

# List representing the original forms of Hindi matras
    original_matras = ['आ', 'इ', 'ई', 'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ', 'ऋ', 'ॲ', 'ऑ', 'अं', 'अ:', 'अं']
    
    hindi_punctuation_marks = ['।', '?', '!', ',', ';', '‘', '’', '(', ')']

# Create a dictionary mapping from actual form to original form
    matras_mapping = dict(zip(actual_matras, original_matras))
    unicode=[]
    for i in range(len(text)):
        if text[i] in hindi_punctuation_marks:
            continue
        
        if text[i] in original_matras or text[i]=='अ':
            unicode.append(text[i])
#             print(text[i])
        elif text[i] in actual_matras:
            original_form = matras_mapping.get(text[i], text[i])  # Use get method to handle cases where actual form not found
            unicode.append(original_form)
#             print(original_form)
        elif text[i]=='्':
            continue
        else:
            if i+1<len(text) and text[i+1] not in actual_matras and text[i+1]!='्':
                tmp=text[i]+'्'
#                 print(tmp)
#                 print('अ')
                unicode.append(tmp)
                unicode.append('अ')
                
            else:
                tmp=text[i]+'्'
#                 print(tmp)
                unicode.append(tmp)
    if text[len(text)-1] not in actual_matras  and text[len(text)-1] not in original_matras and text[len(text)-1]!='।':
#         print('अ')
        unicode.append('अ')
#     print(unicode)
    return unicode
    

In [11]:
import re
def characters_syallables(Tokens):
    # List representing the actual forms of Hindi matras
    actual_matras = ['ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो', 'ौ', 'ृ', 'ॅ', 'ॉ', 'ं', 'ः', 'ँ']

# List representing the original forms of Hindi matras
    original_matras = ['आ', 'इ', 'ई', 'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ', 'ऋ', 'ॲ', 'ऑ', 'अं', 'अ:', 'अं']

# Create a dictionary mapping from actual form to original form
    matras_mapping = dict(zip(actual_matras, original_matras))
    
    hindi_punctuation_marks = ['।', '?', '!', ',', ';', '‘', '’', '(', ')']
    
    Characters_unicode=[]
    syllables_tokens=[]
    Characters_bigram=[]
    bigram_syllables_tokens=[]
    for text in Tokens:
        pattern = r'[\u0900-\u097F]+'
        i= bool(re.match(pattern, text))
        if i!=1:
            continue
        unicode=[]
        bigram=[]
        syllables=[]
        bigram_syllables=[]
        unicode=unic(text)
#         print(unicode)   
        Characters_unicode.append(unicode)
        matras_reverse_mapping = dict(zip(original_matras,actual_matras))
        sy=""
        first_letter=1
        for i in range(len(unicode)):
            if unicode[i] in original_matras or unicode[i]=='अ':
                if first_letter==1:
                    sy=sy+unicode[i]
                    syllables.append(sy)
                    sy=""
                    first_letter=0
                else:
                    if unicode[i]!='अ' and sy!="":
                        correct_matra=matras_reverse_mapping.get(unicode[i], unicode[i])
                        sy=sy+correct_matra
                    elif sy=="":
                        sy=sy+unicode[i]
                    syllables.append(sy)
                    sy=""
                    first_letter=0

            else:
                if i+1<len(unicode) and (unicode[i+1] in original_matras or unicode[i+1]=='अ'):
                    t=unicode[i].replace('्', '')
                    sy=sy+t
                else:
                    sy=sy+unicode[i]
                first_letter=0
        syllables_tokens.append(syllables)
        
        for i in range(len(unicode) - 1):
            combined_character = unicode[i] + unicode[i+1]
            bigram.append(combined_character)
        Characters_bigram.append(bigram)
        for i in range(len(syllables) - 1):
            combined_character = syllables[i] + syllables[i+1]
            bigram_syllables.append(combined_character)
        bigram_syllables_tokens.append(bigram_syllables)
        
    return  Characters_unicode,syllables_tokens,Characters_bigram,bigram_syllables_tokens



In [6]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(tokenized_corpus)

bigram_tokens=[]
for i in range(len(tokenized_corpus) - 1):
            combined_character = tokenized_corpus[i] + tokenized_corpus[i+1]
            bigram_tokens.append(combined_character)
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [12]:
from collections import Counter

def frequency_of_characters(list_of_lists):
    # Flatten the list of lists into a single list of characters
    characters = [char for sublist in list_of_lists for char in sublist]
    
    # Count the frequency of each character
    char_freq = Counter(characters)
    
    # Sort the characters based on their frequencies in descending order
    sorted_char_freq = sorted(char_freq.items(), key=lambda x: x[1], reverse=True)
    
    # Convert the sorted list of tuples to a dictionary
    sorted_char_freq_dict = dict(sorted_char_freq)
    
    return sorted_char_freq_dict

In [13]:
from collections import Counter

def frequency_of_Bigram_Tokens(Tokens):
    # Flatten the list of lists into a single list of characters
  
    # Count the frequency of each character
    char_freq = Counter(Tokens)
    
    # Sort the characters based on their frequencies in descending order
    sorted_char_freq = sorted(char_freq.items(), key=lambda x: x[1], reverse=True)
    
    # Convert the sorted list of tuples to a dictionary
    sorted_char_freq_dict = dict(sorted_char_freq)
    
    return sorted_char_freq_dict

In [9]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)
bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)

In [21]:
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())


Top 20 items in char_freq:
अ 7940222
आ 2991109
ए 2318442
क् 2221101
र् 2115631
ई 1460305
इ 1432248
न् 1334418
स् 1283708
अं 1215588
ह् 1133159
म् 1053237
त् 980066
ल् 919917
ओ 896588
प् 805896
य् 753228
व् 624743
द् 607633
उ 587149

Top 20 items in syllable_freq:
र 1100450
अं 1027392
क 635089
न 559848
स 528574
प 401828
के 401058
ल 379607
ए 370304
त 334408
म 324376
ने 321950
आ 318789
का 302279
इ 297961
है 296912
ह 294153
मे 290328
अ 275984
ब 256260

Top 20 items in bigram_char_freq:
र्अ 1288256
अर् 733046
क्अ 649902
न्अ 571887
स्अ 549090
प्अ 419371
क्ए 402155
त्अ 387549
ल्अ 387526
अन् 376820
अह् 376395
म्अ 358255
एअं 351709
आर् 350628
न्ए 322217
क्आ 307372
ह्अ 297150
ह्ऐ 297061
य्अ 294018
अक् 293414

Top 20 items in bigram_syllable_freq:
मेअं 258780
कर 163123
और 117337
पर 103642
इस 86248
ओअं 80827
हैअं 66265
एक 55966
हीअं 55242
लिए 53591
नही 49103
अप 45854
ड़ 43095
कार 43042
किया 36830
कहा 32993
रने 32619
सर 32486
योअं 32318
यह 31733

Top 20 items in bigram_token_freq:
है। 90674
केलिए 4

In [22]:
# print(char_freq)
# print(syllable_freq)
# print(bigram_char_freq)
# print(bigram_syllable_freq)
# print("hi")
# print(bigram_token_freq)

BPE Model(Vocab=1000)

In [5]:
# Implement BPE with 1000 vocabulary Size


import sentencepiece as spm

# Load the Hindi corpus
corpus_file = "hi_100_1.txt"

# Define the path to save the trained BPE model
model_prefix = "hindi_bpe_model"

# Define the vocabulary size (number of subword units)
vocab_size = 1000

# Train the BPE model
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix,vocab_size=1000, model_type="bpe")

# Load the trained BPE model
bpe_model = spm.SentencePieceProcessor()
bpe_model.load(f"{model_prefix}.model")



True

In [6]:
from collections import Counter
with open("hi_100_1.txt", "r", encoding="utf-8") as f: 
    corpus = f.read()

tokenized_corpus = bpe_model.encode(corpus, out_type=str)




for i, token in enumerate(tokenized_corpus):
    if token.startswith("▁"):
        tokenized_corpus[i] = token[1:].replace("▁", "")
        
# for i in tokenized_corpus:
#     print(i)
token_counter = Counter(tokenized_corpus)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)

i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1

के - 364434
। - 276966
में - 252765
न - 234722
की - 227551
म - 223876
क - 221806
है - 215519
ने - 209162
त - 200426
ल - 200164
प - 195055
, - 191476
स - 187952
र - 175155
व - 173069
से - 171984
ज - 171762
को - 170953
ब - 164575


In [17]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(tokenized_corpus)

bigram_tokens=[]
for i in range(len(tokenized_corpus) - 1):
            combined_character = tokenized_corpus[i] + tokenized_corpus[i+1]
            bigram_tokens.append(combined_character)
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [18]:
# # char_freq=frequency_of_characters(Char)
# syllable_freq=frequency_of_characters (syllable)
# bigram_char_freq=frequency_of_characters(bigram_char)
# bigram_syllable_freq=frequency_of_characters (bigram_syllable)
# bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)

In [19]:
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())

Top 20 items in char_freq:
अ 10127874
आ 2991109
ए 2318442
क् 2221101
र् 2115631
ई 1460305
इ 1432973
न् 1334418
स् 1283708
अं 1215588
ह् 1133159
म् 1053237
त् 980066
ल् 919917
ओ 896588
प् 805896
य् 753228
व् 624743
द् 607633
उ 587149

Top 20 items in syllable_freq:
र 1370674
अं 1119041
क 737317
आ 703406
इ 681207
स 646019
न 640770
ए 548077
प 517389
ल 488579
म 488175
त 451418
के 393586
अ 364231
ह 358310
ब 333018
ग 331524
ने 325957
ज 299260
ओ 296412

Top 20 items in bigram_char_freq:
र्अ 1560975
क्अ 743239
स्अ 660764
न्अ 646731
अर् 545257
प्अ 522609
म्अ 520515
ल्अ 488579
त्अ 482417
क्ए 393586
ह्अ 362715
आर् 360707
एअं 351357
य्अ 347502
ब्अ 333018
ग्अ 331524
न्ए 325957
अह् 313382
व्अ 304077
ज्अ 303331

Top 20 items in bigram_syllable_freq:
मेअं 256824
कर 180284
और 123956
ओअं 123316
पर 112496
इस 100171
हैअं 80599
एअं 79457
अअ 72100
आर 68605
इक 67975
इत 66185
कार 60728
हीअं 56185
अंग 54775
एक 54352
लिए 53777
आन 50796
अप 47971
नही 47279

Top 20 items in bigram_token_freq:
है। 91399
केलिए 44391

In [20]:
# print(char_freq)
# print(syllable_freq)
# print(bigram_char_freq)
# print(bigram_syllable_freq)
# print("hi")
# print(bigram_token_freq)

BPE Model(Vocab=2000)

In [4]:
# Implement BPE with 1000 vocabulary Size


import sentencepiece as spm

# Load the Hindi corpus
corpus_file = "hi_100_1.txt"

# Define the path to save the trained BPE model
model_prefix = "hindi_bpe_2000_model"


# Train the BPE model
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix,vocab_size=2000, model_type="bpe")




sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: hi_100_1.txt
  input_format: 
  model_prefix: hindi_bpe_2000_model
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise

In [6]:
from collections import Counter
# Load the trained BPE model
bpe_model = spm.SentencePieceProcessor()
bpe_model.load(f"{model_prefix}.model")

with open("hi_100_1.txt", "r", encoding="utf-8") as f:
    corpus = f.read()
    


tokenized_corpus = bpe_model.encode(corpus, out_type=str)
for i, token in enumerate(tokenized_corpus):
    if token.startswith("▁"):
        tokenized_corpus[i] = token[1:].replace("▁", "")

# for i in tokenized_corpus:
#     print(i)

token_counter = Counter(tokenized_corpus)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)

i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1

के - 344230
। - 276966
में - 245494
है - 215519
की - 214647
, - 189721
ने - 178508
को - 164642
से - 161960
का - 147974
क - 133331
. - 128642
न - 124503
और - 115365
म - 114131
स - 111941
पर - 105525
प - 99336
व - 96389
ल - 95954


In [7]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(tokenized_corpus)

bigram_tokens=[]
for i in range(len(tokenized_corpus) - 1):
            combined_character = tokenized_corpus[i] + tokenized_corpus[i+1]
            bigram_tokens.append(combined_character)
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [ ]:
till here

In [11]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)
bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)


In [12]:
# print(char_freq)
# print(syllable_freq)
# print(bigram_char_freq)
# print(bigram_syllable_freq)
# print("hi")
# print(bigram_token_freq)
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())

Top 20 items in char_freq:
अ 9126450
आ 2991109
ए 2318442
क् 2221101
र् 2115631
ई 1460305
इ 1432973
न् 1334418
स् 1283708
अं 1215588
ह् 1133159
म् 1053237
त् 980066
ल् 919917
ओ 896588
प् 805896
य् 753228
व् 624743
द् 607633
उ 587149

Top 20 items in syllable_freq:
र 1234034
अं 1089771
क 678758
न 601664
स 599332
आ 518713
इ 518462
प 460937
ल 435541
ए 434637
के 402872
म 400455
त 388356
ह 334884
अ 329072
ने 325957
का 304314
ब 298772
है 296118
ग 294302

Top 20 items in bigram_char_freq:
र्अ 1430396
क्अ 687818
अर् 635321
स्अ 615810
न्अ 607625
प्अ 472653
म्अ 440553
ल्अ 439486
त्अ 427056
क्ए 402872
आर् 366396
एअं 351357
अह् 349873
ह्अ 337544
न्ए 325957
य्अ 325819
क्आ 304314
ब्अ 298772
ह्ऐ 296118
ग्अ 296031

Top 20 items in bigram_syllable_freq:
मेअं 259459
कर 173835
और 119313
पर 109167
ओअं 98838
इस 97589
हैअं 80599
एअं 57064
हीअं 56185
एक 54352
लिए 53777
कार 51507
अअ 50746
नही 49103
इत 46809
आर 46546
अप 46073
इक 45433
ड़ 43488
अंग 40905

Top 20 items in bigram_token_freq:
है। 91399
केलिए 43665


In [ ]:
mBERT model (max_length=1000)

In [2]:
pip install transformers

     ---------------------------------------- 8.4/8.4 MB 15.8 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 kB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00
     ------------------------------------- 330.1/330.1 kB 10.3 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 23.2 MB/s eta 0:00:00
     ---------------------------------------- 170.9/170.9 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import BertTokenizer, TFBertModel
from collections import Counter

corpus_file = "hi_100_1.txt"
# Load the mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased",max_length=1000)



# Tokenize the Hindi sentence


with open(corpus_file, "r", encoding="utf-8") as f:
    corpus = f.read()
# Print the tokenized output

tokenized_corpus = tokenizer.tokenize(corpus)

for i, token in enumerate(tokenized_corpus):
    if token.startswith("#"):
        tokenized_corpus[i] = token[1:].replace("#", "")
token_counter = Counter(tokenized_corpus)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)

i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1


के - 347164
स - 330691
प - 314836
म - 305742
र - 301686
। - 276966
ब - 259624
में - 246133
न - 235575
है - 215601
की - 213189
ा - 205239
क - 193113
, - 191460
ल - 187736
को - 173966
ी - 173352
ने - 172673
से - 165303
का - 164733


In [10]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(tokenized_corpus)

bigram_tokens=[]
for i in range(len(tokenized_corpus) - 1):
            combined_character = tokenized_corpus[i] + tokenized_corpus[i+1]
            bigram_tokens.append(combined_character)
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [11]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)
bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)

In [12]:
# print(char_freq)
# print(syllable_freq)
# print(bigram_char_freq)
# print(bigram_syllable_freq)
# print("hi")
# print(bigram_token_freq)
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())

Top 20 items in char_freq:
अ 10630804
आ 2990653
ए 2317707
क् 2220811
र् 2115389
ई 1460093
इ 1432830
न् 1334247
स् 1283571
अं 1215462
ह् 1133069
म् 1053101
त् 979855
ल् 919790
ओ 896299
प् 805642
य् 753100
व् 624631
द् 607566
उ 586985

Top 20 items in syllable_freq:
र 1519633
अं 1120017
क 772941
स 758570
न 739490
आ 722192
इ 716033
प 605704
ए 573911
म 546678
ल 496830
त 439358
अ 436636
उ 411143
के 397142
ब 382913
ह 352607
ग 348705
ओ 338416
ई 332946

Top 20 items in bigram_char_freq:
र्अ 1633762
क्अ 776956
स्अ 764871
न्अ 742437
प्अ 610571
अर् 598163
म्अ 563333
ल्अ 497419
त्अ 495102
क्ए 397142
ब्अ 383609
ह्अ 352837
ग्अ 352622
य्अ 338692
एअं 322613
व्अ 310488
न्ए 306789
आर् 304446
ज्अ 301061
क्आ 299923

Top 20 items in bigram_syllable_freq:
मेअं 254882
कर 205213
अअ 133611
पर 127688
और 125194
इस 119334
ओअं 118355
हैअं 80610
एक 67768
अंग 54850
लिए 53250
एअं 51529
नही 47362
हीअं 47362
कार 47089
ड़ 45035
आर 44496
उन 44146
इन 42615
इक 41972

Top 20 items in bigram_token_freq:
है। 91400
केलिए 44090

In [5]:
from transformers import BertTokenizer, TFBertModel
from collections import Counter

corpus_file = "hi_100_1.txt"
# Load the mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased",max_length=2000)



# Tokenize the Hindi sentence


with open(corpus_file, "r", encoding="utf-8") as f:
    corpus = f.read()
# Print the tokenized output

tokenized_corpus = tokenizer.tokenize(corpus)

for i, token in enumerate(tokenized_corpus):
    if token.startswith("#"):
        tokenized_corpus[i] = token[1:].replace("#", "")
token_counter = Counter(tokenized_corpus)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)

i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1


/Users/tanmaydubey/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/tanmaydubey/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


के - 347164
स - 330691
प - 314836
म - 305742
र - 301686
। - 276966
ब - 259624
में - 246133
न - 235575
है - 215601
की - 213189
ा - 205239
क - 193113
, - 191460
ल - 187736
को - 173966
ी - 173352
ने - 172673
से - 165303
का - 164733


In [6]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(tokenized_corpus)

bigram_tokens=[]
for i in range(len(tokenized_corpus) - 1):
            combined_character = tokenized_corpus[i] + tokenized_corpus[i+1]
            bigram_tokens.append(combined_character)
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [7]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)
bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)

In [8]:
# print(char_freq)
# print(syllable_freq)
# print(bigram_char_freq)
# print(bigram_syllable_freq)
# print("hi")
# print(bigram_token_freq)
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())

Top 20 items in char_freq:
अ 10630804
आ 2990653
ए 2317707
क् 2220811
र् 2115389
ई 1460093
इ 1432830
न् 1334247
स् 1283571
अं 1215462
ह् 1133069
म् 1053101
त् 979855
ल् 919790
ओ 896299
प् 805642
य् 753100
व् 624631
द् 607566
उ 586985

Top 20 items in syllable_freq:
र 1519633
अं 1120017
क 772941
स 758570
न 739490
आ 722192
इ 716033
प 605704
ए 573911
म 546678
ल 496830
त 439358
अ 436636
उ 411143
के 397142
ब 382913
ह 352607
ग 348705
ओ 338416
ई 332946

Top 20 items in bigram_char_freq:
र्अ 1633762
क्अ 776956
स्अ 764871
न्अ 742437
प्अ 610571
अर् 598163
म्अ 563333
ल्अ 497419
त्अ 495102
क्ए 397142
ब्अ 383609
ह्अ 352837
ग्अ 352622
य्अ 338692
एअं 322613
व्अ 310488
न्ए 306789
आर् 304446
ज्अ 301061
क्आ 299923

Top 20 items in bigram_syllable_freq:
मेअं 254882
कर 205213
अअ 133611
पर 127688
और 125194
इस 119334
ओअं 118355
हैअं 80610
एक 67768
अंग 54850
लिए 53250
एअं 51529
नही 47362
हीअं 47362
कार 47089
ड़ 45035
आर 44496
उन 44146
इन 42615
इक 41972

Top 20 items in bigram_token_freq:
है। 91400
केलिए 44090

White Space TokeniZer

In [9]:
def whitespace_tokenizer(text):
    tokens = text.split()
    return tokens



corpus_file = "hi_100_1.txt"
# Example usage:


with open(corpus_file, "r", encoding="utf-8") as f:
    corpus = f.read()
# Print the tokenized output

tokenized_corpus = whitespace_tokenizer(corpus)
token_counter = Counter(tokenized_corpus)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)

i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1


के - 316314
में - 237426
की - 189769
को - 145266
से - 136979
और - 114563
का - 109743
ने - 102378
पर - 88560
है। - 88260
कि - 76741
है - 72656
भी - 63920
एक - 49279
लिए - 47955
इस - 47558
कर - 44563
नहीं - 44168
ही - 40104
तो - 33469


In [12]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(tokenized_corpus)

bigram_tokens=[]
for i in range(len(tokenized_corpus) - 1):
            combined_character = tokenized_corpus[i] + tokenized_corpus[i+1]
            bigram_tokens.append(combined_character)
            
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [13]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)
bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)


In [14]:
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())

Top 20 items in char_freq:
अ 7528720
आ 2969906
ए 2303784
क् 2211430
र् 2100454
ई 1444291
इ 1422144
न् 1324990
स् 1272104
अं 1206650
ह् 1128196
म् 1043970
त् 973846
ल् 913027
ओ 891832
प् 799153
य् 747074
व् 617371
द् 603397
उ 582769

Top 20 items in syllable_freq:
र 1000776
अं 995476
क 595485
न 500060
स 484209
अ 438743
के 404384
प 388026
ने 327295
ल 322984
का 306879
त 306651
है 296934
मे 291883
म 288959
ए 282985
ह 275903
ब 241243
की 235042
या 224715

Top 20 items in bigram_char_freq:
र्अ 1162998
अर् 782283
क्अ 607330
स्अ 512652
न्अ 510676
अन् 431480
क्ए 406484
प्अ 401732
अह् 390377
आर् 365328
एअं 358590
अक् 352595
त्अ 351391
न्ए 328129
ल्अ 326319
म्अ 319386
क्आ 313177
अत् 310110
ह्ऐ 297111
य्आ 296448

Top 20 items in bigram_syllable_freq:
मेअं 259109
कर 159692
और 115492
पर 99454
इस 82257
हैअं 80754
हीअं 55743
एक 54004
लिए 53993
नही 49289
अप 44969
कार 38812
किया 37419
योअं 34672
रने 34477
कहा 33092
यह 31031
गया 30277
सर 29877
उन 28847

Top 20 items in bigram_token_freq:
केलिए 42448
हैकि 

# Indic Bert(max_length=1000)

In [1]:
import sentencepiece as spm
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer

In [2]:
# Load the model
model = AutoModel.from_pretrained('ai4bharat/indic-bert')
# Load the tokenizer
tokenizer= AutoTokenizer.from_pretrained('ai4bharat/indic-bert',max_length=1000, truncation=True)

/Users/tanmaydubey/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
from collections import Counter
# with open("hi_100_1.txt", "r", encoding="utf-8") as f:
#     corpus = f.read()


tokenized_corpus=[]
with open("hi_100_1.txt", "r", encoding="utf-8") as f:
    for line in f:
        line_tokens= tokenizer.tokenize(line.strip())
        tokenized_corpus.append(line_tokens)





In [5]:
flattened_list_tokens = [item for sublist in tokenized_corpus for item in sublist]

In [6]:
for i, token in enumerate(flattened_list_tokens):
    if token.startswith("▁"):
        flattened_list_tokens[i] = token[1:].replace("▁", "")


token_counter = Counter(flattened_list_tokens)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)


i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1

क - 1148482
ह - 528778
य - 451193
न - 415590
स - 388084
म - 359008
त - 314391
पर - 278853
। - 274203
 - 262368
कर - 208188
द - 187350
, - 185232
ल - 171856
व - 164856
ग - 158229
ज - 156342
र - 146564
थ - 139602
ए - 136760


In [14]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(flattened_list_tokens)

bigram_tokens=[]
for i in range(len(flattened_list_tokens) - 1):
            combined_character = flattened_list_tokens[i] + flattened_list_tokens[i+1]
            bigram_tokens.append(combined_character)
            
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [15]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)
bigram_token_freq=frequency_of_Bigram_Tokens(bigram_tokens)

In [16]:
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

print("\nTop 20 items in bigram_token_freq:")
print_top_5(bigram_token_freq.items())

Top 20 items in char_freq:
अ 18133834
क् 2221101
र् 2115912
न् 1335232
स् 1283708
ह् 1133159
म् 1053237
त् 980066
ल् 919917
प् 805896
य् 753228
व् 624743
द् 607633
ज् 556610
ब् 524636
ग् 479960
ट् 318634
श् 297486
ए 286325
च् 272542

Top 20 items in syllable_freq:
क 2221101
र 2115912
न 1335232
स 1283708
ह 1133159
म 1053237
त 980066
ल 919917
प 805896
य 753228
व 624743
द 607633
ज 556610
ब 524636
ग 479960
ट 318634
श 297486
ए 286325
अ 277021
च 272542

Top 20 items in bigram_char_freq:
क्अ 2221101
र्अ 2115912
अर् 1539100
न्अ 1335232
स्अ 1283708
ह्अ 1133159
म्अ 1053237
त्अ 980066
ल्अ 919917
प्अ 805896
य्अ 753228
अन् 692776
व्अ 624743
द्अ 607633
ज्अ 556610
अल् 525756
ब्अ 524636
ग्अ 479960
अत् 422482
अह् 410505

Top 20 items in bigram_syllable_freq:
कर 331433
पर 311493
और 115918
इस 100946
तर 94887
रह 93236
वर 91167
दर 81145
रत 81053
सम 73031
सर 69826
गर 69626
जन 67623
लग 63926
मन 62259
एक 60681
बर 59280
पन 58532
उन 57239
मर 55877

Top 20 items in bigram_token_freq:
ह। 129653
नह 61175
तह 59893


In [ ]:
Indic_BERT(max_length=2000)

In [17]:
tokenizer= AutoTokenizer.from_pretrained('ai4bharat/indic-bert',max_length=2000, truncation=True)




In [18]:
from collections import Counter
# with open("hi_100_1.txt", "r", encoding="utf-8") as f:
#     corpus = f.read()


tokenized_corpus=[]
with open("hi_100_1.txt", "r", encoding="utf-8") as f:
    for line in f:
        line_tokens= tokenizer.tokenize(line.strip())
        tokenized_corpus.append(line_tokens)

In [19]:
flattened_list_tokens = [item for sublist in tokenized_corpus for item in sublist]

In [20]:
for i, token in enumerate(flattened_list_tokens):
    if token.startswith("▁"):
        flattened_list_tokens[i] = token[1:].replace("▁", "")


token_counter = Counter(flattened_list_tokens)

# Store tokens along with their frequencies
tokens_with_frequencies = [(token, freq) for token, freq in token_counter.items()]

# Sort tokens by frequency in descending order
tokens_with_frequencies.sort(key=lambda x: x[1], reverse=True)


i=1
# Print tokens with frequencies
for token, freq in tokens_with_frequencies:
    if i<=20:
        print(token, "-", freq)
        i=i+1


क - 1148482
ह - 528778
य - 451193
न - 415590
स - 388084
म - 359008
त - 314391
पर - 278853
। - 274203
 - 262368
कर - 208188
द - 187350
, - 185232
ल - 171856
व - 164856
ग - 158229
ज - 156342
र - 146564
थ - 139602
ए - 136760


In [21]:
Char,syllable,bigram_char,bigram_syllable=characters_syallables(flattened_list_tokens)

bigram_tokens=[]
for i in range(len(flattened_list_tokens) - 1):
            combined_character = flattened_list_tokens[i] + flattened_list_tokens[i+1]
            bigram_tokens.append(combined_character)
            
# print(Char)
# print(syllable)
# print(bigram_char)
# print(bigram_syllable)
# print(bigram_tokens)

In [23]:
char_freq=frequency_of_characters(Char)
syllable_freq=frequency_of_characters (syllable)
bigram_char_freq=frequency_of_characters(bigram_char)
bigram_syllable_freq=frequency_of_characters (bigram_syllable)


In [ ]:
def print_top_5(dictionary):
    count = 0
    for key, value in dictionary:
        if count >= 20:
            break
        print(key, value)
        count += 1


print("Top 20 items in char_freq:")
print_top_5(char_freq.items())

print("\nTop 20 items in syllable_freq:")
print_top_5(syllable_freq.items())

print("\nTop 20 items in bigram_char_freq:")
print_top_5(bigram_char_freq.items())

print("\nTop 20 items in bigram_syllable_freq:")
print_top_5(bigram_syllable_freq.items())

